In [1]:
import pandas as pd
from transformers import BartForConditionalGeneration, BartTokenizer, T5ForConditionalGeneration, T5Tokenizer, PegasusForConditionalGeneration, PegasusTokenizer
from rouge import Rouge
import warnings

warnings.filterwarnings("ignore")

# Load the BBC news data (taken from: https://www.kaggle.com/datasets/pariza/bbc-news-summary)
df = pd.read_csv('bbc_news.csv')

# Considering 20 rows from the top for text summarization
data = df.head(20)
data

C:\Users\anikd\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
C:\Users\anikd\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


,Articles,Summaries
0,Ad sales boost Time Warner profit..Quarterly p...,TimeWarner said fourth quarter sales rose 2% t...
1,Dollar gains on Greenspan speech..The dollar h...,The dollar has hit its highest level against t...
2,Yukos unit buyer faces loan claim..The owners ...,Yukos' owner Menatep Group says it will ask Ro...
3,High fuel prices hit BA's profits..British Air...,"Rod Eddington, BA's chief executive, said the ..."
4,Pernod takeover talk lifts Domecq..Shares in U...,Pernod has reduced the debt it took on to fund...
5,Japan narrowly escapes recession..Japan's econ...,"On an annual basis, the data suggests annual g..."
6,Jobs growth still slow in the US..The US creat...,The job gains mean that President Bush can cel...
7,"India calls for fair trade rules..India, which...",At a conference on developing enterprise hoste...
8,Ethiopia's crop production up 24%..Ethiopia pr...,"In 2003, crop production totalled 11.49 millio..."
9,Court rejects $280bn tobacco case..A US govern...,A US government claim accusing the country's b...


In [2]:
max_length_input = 1024
max_length_output = 150
num_beams = 8


# Text summarization function using T5
def summarize_text_t5(text):
    tokenizer = T5Tokenizer.from_pretrained('t5-base')
    model = T5ForConditionalGeneration.from_pretrained('t5-base')

    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=max_length_input, truncation=True)
    outputs = model.generate(inputs, max_length=max_length_output, num_beams=num_beams, early_stopping=True)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return summary


# Text summarization function using BART
def summarize_text_bart(text):
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
    model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')
    
    input_ids = tokenizer.encode(text, return_tensors='pt', max_length=max_length_input, truncation=True)
    summary_ids = model.generate(input_ids, max_length=max_length_output, num_beams=num_beams, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return summary


# Text summarization function using Pegasus
def summarize_text_pegasus(text):
    tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-large')
    model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-large')

    inputs = tokenizer([text], max_length=max_length_input, return_tensors='pt', truncation=True)
    summary_ids = model.generate(inputs['input_ids'], max_length=max_length_output, num_beams=num_beams, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return summary


# Function for calculating ROUGE scores
def calculate_rouge_scores(model_names, generated_summaries, references):
    rouge_scores = {}
    rouge = Rouge()
    
    for model_name, generated_summary in zip(model_names, generated_summaries):
        scores = rouge.get_scores(generated_summary, references, avg=True)
        rouge_scores[model_name] = scores
        
    return rouge_scores


# Model names and their corresponding text summarization functions
models = {'T5': summarize_text_t5, 
          'BART': summarize_text_bart, 
          'Pegasus': summarize_text_pegasus
         }

# Summarize texts using the T5, BART and Pegasus
model_names = list(models.keys())
generated_summaries = {model_name: [] for model_name in model_names}

for model_name, model_function in models.items():
    data[model_name + '_summaries'] = data['Articles'].apply(lambda x: model_function(x))
    generated_summaries[model_name] = data[model_name + '_summaries'].tolist()

references = data['Summaries'].tolist()
rouge_scores = calculate_rouge_scores(model_names, generated_summaries.values(), references)

print("ROUGE Scores:")
for model_name, scores in rouge_scores.items():
    print(f"{model_name}:")
    for metric, score in scores.items():
        print(f"\t{metric}: {score}")

print("\nDataFrame with Summaries:")
data

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.encoder.embed_p

ROUGE Scores:
T5:
	rouge-1: {'r': 0.23237135256046124, 'p': 0.6381656856900066, 'f': 0.3345984806673038}
	rouge-2: {'r': 0.13408391333069516, 'p': 0.4316913999877915, 'f': 0.19983009185887507}
	rouge-l: {'r': 0.2281291839385212, 'p': 0.6254346685519687, 'f': 0.32836893082794677}
BART:
	rouge-1: {'r': 0.5195894083346405, 'p': 0.5221021432434262, 'f': 0.5074633471893999}
	rouge-2: {'r': 0.3665849503954596, 'p': 0.37091480926620224, 'f': 0.3574846286580232}
	rouge-l: {'r': 0.5075290588615518, 'p': 0.510038934616644, 'f': 0.49579763124903564}
Pegasus:
	rouge-1: {'r': 0.3541593859634691, 'p': 0.696601253378674, 'f': 0.4617342712495206}
	rouge-2: {'r': 0.24753986058393251, 'p': 0.5552176467859821, 'f': 0.33592491652500817}
	rouge-l: {'r': 0.34595365712544707, 'p': 0.678475714480546, 'f': 0.4507406942701689}

DataFrame with Summaries:


,Articles,Summaries,T5_summaries,BART_summaries,Pegasus_summaries
0,Ad sales boost Time Warner profit..Quarterly p...,TimeWarner said fourth quarter sales rose 2% t...,time Warner profits up 76% to $1.13bn (£600m) ...,Ad sales boost Time Warner profit..Quarterly p...,Ad sales boost Time Warner profit..Quarterly p...
1,Dollar gains on Greenspan speech..The dollar h...,The dollar has hit its highest level against t...,the dollar has hit its highest level against t...,Dollar gains on Greenspan speech..The dollar h...,Dollar gains on Greenspan speech..The dollar h...
2,Yukos unit buyer faces loan claim..The owners ...,Yukos' owner Menatep Group says it will ask Ro...,the owners of embattled Russian oil giant are ...,Yukos unit buyer faces loan claim..The owners ...,Yukos unit buyer faces loan claim..The owners ...
3,High fuel prices hit BA's profits..British Air...,"Rod Eddington, BA's chief executive, said the ...",the airline made a pre-tax profit of £75m ($14...,High fuel prices hit BA's profits..British Air...,BA's profits were still better than market exp...
4,Pernod takeover talk lifts Domecq..Shares in U...,Pernod has reduced the debt it took on to fund...,Allied Domecq shares in London rose 4% by 1200...,Pernod takeover talk lifts DomecqShares in UK ...,Pernod said it was seeking acquisitions but re...
5,Japan narrowly escapes recession..Japan's econ...,"On an annual basis, the data suggests annual g...",the economy teetered on the brink of a technic...,Japan narrowly escapes recession..Japan's econ...,Japan narrowly escapes recession..Japan's econ...
6,Jobs growth still slow in the US..The US creat...,The job gains mean that President Bush can cel...,a fall in jobseekers pushes the unemployment r...,Jobs growth still slow in the US..The US creat...,"However, when adding December and January's fi..."
7,"India calls for fair trade rules..India, which...",At a conference on developing enterprise hoste...,india's finance minister lashed out at restric...,"India calls for fair trade rules..India, which...","India calls for fair trade rules..India, which..."
8,Ethiopia's crop production up 24%..Ethiopia pr...,"In 2003, crop production totalled 11.49 millio...","in 2003, crop production totalled 11.49 millio...",The report calculated emergency food requireme...,Ethiopia's crop production up 24%..Ethiopia pr...
9,Court rejects $280bn tobacco case..A US govern...,A US government claim accusing the country's b...,the demand for $280bn (£155bn) - filed by the ...,Court rejects $280bn tobacco case..A US govern...,Court rejects $280bn tobacco case..A US govern...


In [3]:
pd.set_option('display.max_colwidth', 350)

# Show the summary by T5
t5 = data[['Articles', 'T5_summaries']]
t5

,Articles,T5_summaries
0,"Ad sales boost Time Warner profit..Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (Â£600m) for the three months to December, from $639m year-earlier...The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarte...",time Warner profits up 76% to $1.13bn (£600m) for the three months to December. profit buoyed by high-speed internet connections and higher advert sales. company now owns 8% of search-engine google.
1,Dollar gains on Greenspan speech..The dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise...And Alan Greenspan highlighted the US government's willingness to curb spending and rising household savings as factors which may help to reduce it. In late...,the dollar has hit its highest level against the euro in almost three months. the dollar tumbled on the back of worse-than-expected US jobs data. greenspan said the current account deficit is set to stabilise.
2,Yukos unit buyer faces loan claim..The owners of embattled Russian oil giant Yukos are to ask the buyer of its former production unit to pay back a $900m (Â£479m) loan...State-owned Rosneft bought the Yugansk unit for $9.3bn in a sale forced by Russia to part settle a $27.5bn tax claim against Yukos. Yukos' owner Menatep Group says it will ask ...,the owners of embattled Russian oil giant are to ask the buyer of its former production unit to pay back a $900m loan. state-owned Rosneft bought the Yugansk unit for $9.3bn in a sale forced by Russia to part settle a $27.5bn tax claim against Yukos.
3,"High fuel prices hit BA's profits..British Airways has blamed high fuel prices for a 40% drop in profits...Reporting its results for the three months to 31 December 2004, the airline made a pre-tax profit of Â£75m ($141m) compared with Â£125m a year earlier. Rod Eddington, BA's chief executive, said the results were ""respectable"" in a third qua...",the airline made a pre-tax profit of £75m ($141m) compared with £125m a year earlier. fuel costs rose by £106m or 47.3% in a third quarter. BA's profits were still better than market expectation of £59m.
4,"Pernod takeover talk lifts Domecq..Shares in UK drinks and food firm Allied Domecq have risen on speculation that it could be the target of a takeover by France's Pernod Ricard...Reports in the Wall Street Journal and the Financial Times suggested that the French spirits firm is considering a bid, but has yet to contact its target. Allied Domec...","Allied Domecq shares in London rose 4% by 1200 GMT, while pernod shares in Paris slipped 1.2%. pernod said it was seeking acquisitions but refused to comment on specifics."
5,"Japan narrowly escapes recession..Japan's economy teetered on the brink of a technical recession in the three months to September, figures show...Revised figures indicated growth of just 0.1% - and a similar-sized contraction in the previous quarter. On an annual basis, the data suggests annual growth of just 0.2%, suggesting a much more hesita...","the economy teetered on the brink of a technical recession in the three months to September. on an annual basis, the data suggests annual growth of just 0.2%."
6,"Jobs growth still slow in the US..The US created fewer jobs than expected in January, but a fall in jobseekers pushed the unemployment rate to its lowest level in three years...According to Labor Department figures, US firms added only 146,000 jobs in January. The gain in non-farm payrolls was below market expectations of 190,000 new jobs. Neve...",a fall in jobseekers pushes the unemployment rate to its lowest level in three years. president bush can celebrate a net growth in jobs in his first term in office.
7,"India calls for fair trade rules..India, which attends the G7 meeting of seven leading industrialised nations on Friday, is unlikely t

In [4]:
# Show the summary by BART
bart = data[['Articles', 'BART_summaries']]
bart

,Articles,BART_summaries
0,"Ad sales boost Time Warner profit..Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (Â£600m) for the three months to December, from $639m year-earlier...The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarte...","Ad sales boost Time Warner profit..Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (Â£600m) for the three months to December, from $639m year-earlier...The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. Its profits were buoyed by on..."
1,Dollar gains on Greenspan speech..The dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise...And Alan Greenspan highlighted the US government's willingness to curb spending and rising household savings as factors which may help to reduce it. In late...,Dollar gains on Greenspan speech..The dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise...And Alan Greenspan highlighted the US government's willingness to curb spending and rising household savings as factors which may help to reduce it. On Frid...
2,Yukos unit buyer faces loan claim..The owners of embattled Russian oil giant Yukos are to ask the buyer of its former production unit to pay back a $900m (Â£479m) loan...State-owned Rosneft bought the Yugansk unit for $9.3bn in a sale forced by Russia to part settle a $27.5bn tax claim against Yukos. Yukos' owner Menatep Group says it will ask ...,Yukos unit buyer faces loan claim..The owners of embattled Russian oil giant Yukos are to ask the buyer of its former production unit to pay back a $900m (Â£479m) loan...State-owned Rosneft bought the Yugansk unit for $9.3bn in a sale forced by Russia to part settle a $27.5bn tax claim against Yukos. Yukos claims its downfall was punishment for...
3,"High fuel prices hit BA's profits..British Airways has blamed high fuel prices for a 40% drop in profits...Reporting its results for the three months to 31 December 2004, the airline made a pre-tax profit of Â£75m ($141m) compared with Â£125m a year earlier. Rod Eddington, BA's chief executive, said the results were ""respectable"" in a third qua...","High fuel prices hit BA's profits..British Airways has blamed high fuel prices for a 40% drop in profits...Reporting its results for the three months to 31 December 2004, the airline made a pre-tax profit of Â£75m ($141m) compared with Â**125m a year earlier. Turnover for the quarter was up 4.3% to ÂÂ£1.97bn, further benefiting from a rise in c..."
4,"Pernod takeover talk lifts Domecq..Shares in UK drinks and food firm Allied Domecq have risen on speculation that it could be the target of a takeover by France's Pernod Ricard...Reports in the Wall Street Journal and the Financial Times suggested that the French spirits firm is considering a bid, but has yet to contact its target. Allied Domec...","Pernod takeover talk lifts DomecqShares in UK drinks and food firm Allied Domecqs have risen on speculation that it could be the target of a takeover by France's Pernod Ricard...Reports in the Wall Street Journal and the Financial Times suggested that the French spirits firm is considering a bid, but has yet to contact its target. The WSJ said ..."
5,"Japan narrowly escapes recession..Japan's economy teetered on the brink of a technical recession in the three months to September, figures show...Revised figures indicated growth of just 0.1% - and a similar-sized contraction in the previous quarter. On an annual basis, the data suggests annual growth of just 0.2%, suggesting a much more hesita...","Japan narrowly escapes recession..Japan's economy teetered on the brink of a technical recession

In [5]:
# Show the summary by Pegasus
pegasus = data[['Articles', 'Pegasus_summaries']]
pegasus

,Articles,Pegasus_summaries
0,"Ad sales boost Time Warner profit..Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (Â£600m) for the three months to December, from $639m year-earlier...The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarte...","Ad sales boost Time Warner profit..Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier...The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner also has to restate..."
1,Dollar gains on Greenspan speech..The dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise...And Alan Greenspan highlighted the US government's willingness to curb spending and rising household savings as factors which may help to reduce it. In late...,"Dollar gains on Greenspan speech..The dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise...And Alan Greenspan highlighted the US government's willingness to curb spending and rising household savings as factors which may help to reduce it. ""I thin..."
2,Yukos unit buyer faces loan claim..The owners of embattled Russian oil giant Yukos are to ask the buyer of its former production unit to pay back a $900m (Â£479m) loan...State-owned Rosneft bought the Yugansk unit for $9.3bn in a sale forced by Russia to part settle a $27.5bn tax claim against Yukos. Yukos' owner Menatep Group says it will ask ...,Yukos unit buyer faces loan claim..The owners of embattled Russian oil giant Yukos are to ask the buyer of its former production unit to pay back a $900m (£479m) loan...State-owned Rosneft bought the Yugansk unit for $9.3bn in a sale forced by Russia to part settle a $27.5bn tax claim against Yukos.
3,"High fuel prices hit BA's profits..British Airways has blamed high fuel prices for a 40% drop in profits...Reporting its results for the three months to 31 December 2004, the airline made a pre-tax profit of Â£75m ($141m) compared with Â£125m a year earlier. Rod Eddington, BA's chief executive, said the results were ""respectable"" in a third qua...","BA's profits were still better than market expectation of £59m, and it expects a rise in full-year revenues...To help offset the increased price of aviation fuel, BA last year introduced a fuel surcharge for passengers...In October, it increased this from £6 to £10 one-way for all long-haul flights, while the short-haul surcharge was raised fro..."
4,"Pernod takeover talk lifts Domecq..Shares in UK drinks and food firm Allied Domecq have risen on speculation that it could be the target of a takeover by France's Pernod Ricard...Reports in the Wall Street Journal and the Financial Times suggested that the French spirits firm is considering a bid, but has yet to contact its target. Allied Domec...","Pernod said it was seeking acquisitions but refused to comment on specifics...Pernod's last major purchase was a third of US giant Seagram in 2000, the move which propelled it into the global top three of drinks firms."
5,"Japan narrowly escapes recession..Japan's economy teetered on the brink of a technical recession in the three months to September, figures show...Revised figures indicated growth of just 0.1% - and a similar-sized contraction in the previous quarter. On an annual basis, the data suggests annual growth of just 0.2%, suggesting a much more hesita...","Japan narrowly escapes recession..Japan's economy teetered on the brink of a technical recession in the three months to September, figures show...Revised figures indicated growth of just 0.1% - and a similar-sized contraction in the previous quarter."
6,"Jobs growth sti